# Weighted calculations

The interface for weighted calculations in `microdf` is as follows:
```
weighted_function(df: pd.DataFrame, col: str, w: str = None):
```
where `col` and `w` are names of columns in `df` representing the column to aggregate and the (optional) weight, respectively.
Some functions take other arguments as well, but these are common to the functions here.

## Setup

In [1]:
import numpy as np
import pandas as pd

import taxcalc as tc
import microdf as mdf

## Load data

Start with a `DataFrame` with `nu18` and `XTOT`, and also calculate `XTOT_m`.

In [2]:
df = mdf.calc_df(group_vars=['nu18'], metric_vars=['XTOT'])
df.columns

Index(['mcare_ben', 'other_ben', 'ssi_ben', 'wic_ben', 'e02400', 'e02300',
       'mcaid_ben', 'snap_ben', 'nu18', 'aftertax_income', 'vet_ben',
       'tanf_ben', 's006', 'XTOT', 'housing_ben', 'expanded_income',
       'market_income', 'bens', 'tax', 's006_m', 'XTOT_m'],
      dtype='object')

From this we can calculate the number of people and tax units by the tax unit's number of children.

In [3]:
df.groupby('nu18')[['s006_m', 'XTOT_m']].sum()

,s006_m,XTOT_m
nu18,,
0.0,152.988772,209.816367
1.0,22.688253,54.115850
2.0,18.859945,68.880292
3.0,7.438481,34.795527
4.0,2.371111,13.539261
5.0,0.744276,5.015182
6.0,0.216158,1.688063
7.0,0.090332,0.790239
8.0,0.026501,0.258552


What if we also want to calculate the total number of *children* by the tax unit's number of children?

For this we can use `add_weighted_metrics`, the function called within `calc_df`.

In [4]:
mdf.add_weighted_metrics(df=df, metric_vars='nu18')

Now we can do the same thing as before, with the new `nu18_m` column.

In [5]:
df.groupby('nu18')[['nu18_m']].sum()

,nu18_m
nu18,
0.0,0.000000
1.0,22.688253
2.0,37.719889
3.0,22.315444
4.0,9.484444
5.0,3.721381
6.0,1.296949
7.0,0.632325
8.0,0.212008


We can also calculate weighted sums without adding the weighted metric.

In [6]:
total_children = mdf.weighted_sum(df=df, col='nu18', w='s006')
'Total children: ' + str(int(round(total_children / 1e6))) + 'M.'

'Total children: 98M.'

We can also calculate the weighted mean and median.

In [7]:
mdf.weighted_mean(df=df, col='nu18', w='s006')

0.4782626894263673

In [8]:
mdf.weighted_median(df=df, col='nu18', w='s006')

0.0

We can also look at a range of quantiles.

In [9]:
decile_bounds = np.arange(0, 1.1, 0.1)
deciles = mdf.weighted_quantile(df=df, col='nu18', w='s006',
                                quantiles=decile_bounds)
pd.DataFrame(deciles, index=decile_bounds)

,0
0.0,0.0
0.1,0.0
0.2,0.0
0.3,0.0
0.4,0.0
0.5,0.0
0.6,0.0
0.7,0.0
0.8,1.0
0.9,2.0
